## Import modules

In [ ]:
import pandas as pd

## Read in and Inspect Data

In [ ]:
moon_df = pd.read_csv('full-moon-calendar-1900-2050/full_moon.csv')
moon_df

In [ ]:
births1_df = pd.read_csv('fivethirtyeight-births-dataset/US_births_1994-2003_CDC_NCHS.csv')
births1_df

In [ ]:
births2_df = pd.read_csv('fivethirtyeight-births-dataset/US_births_2000-2014_SSA.csv')
births2_df

In [ ]:
# Combine birth data into one dataframe
births_df = pd.concat([births1_df, births2_df], ignore_index=True)

In [ ]:
births_df

In [ ]:
# Remove duplicates because the two birth datasets have overlap from 2000-2003

births_df.drop_duplicates()

In [37]:
# Dropping duplicates was unsuccessful - the data must conflict

births1_df.loc[births1_df['year'] == 2001].head()

,year,month,date_of_month,day_of_week,births
2557,2001,1,1,1,7437
2558,2001,1,2,2,10346
2559,2001,1,3,3,12113
2560,2001,1,4,4,12429
2561,2001,1,5,5,12427


In [38]:
births2_df.loc[births2_df['year'] == 2001].head()

,year,month,date_of_month,day_of_week,births
366,2001,1,1,1,7663
367,2001,1,2,2,10635
368,2001,1,3,3,12449
369,2001,1,4,4,12678
370,2001,1,5,5,12647


In [ ]:
# The data does conflict slightly, so the overlapping years  must be dropped from one of the datasets
